7월 11일 학습

- 애플리케이션 보안: 프로토콜 하나하나에 대한 취약점 분석

- 시스템 보안: 
    - 서버 보안(계정 관리, 파일 시스템 보안)
    - 웹서버, FTP 서버 구축 --> 보안 설정

- 네트워크 보안: TCP/UDP,IP 등 전송 계층과 네트워크(인터넷) 계층에 대한 보안

- 오늘 할 내용
    - TCP,UDP,IP,ICMP,ARP에 대한 패킷 분석
    - 와이어샤크(기능 위주)
    - 와이어샤크를 이용한 공격패킷 분석(기능에 대한 지식을 바탕으로)
    - 와이어샤크를 기반한 장애 처리(기능에 대한 지식을 바탕으로)

   
    - TCP 기능
        - 데이터 전송 전: 3 Way handshake(연결 확립)
        - 데이터 전송 중: 신뢰성 확인, 오류 제어, 흐름 제어
        - 데이터 전송 후: 4 Way handshake(termination)(연결 종료)

    

## TCP 
### TCP 기능
- 데이터 전송 전: 3 Way handshake(연결 확립. 서로의 활성화 여부 확인)
- 데이터 전송 중: 신뢰성 확인, 오류 제어, 흐름 제어
- 데이터 전송 후: 4 Way handshake(termination)(연결 종료)

### TCP 헤더
- 헤더는 기본 20에서 최대 60바이트이다.(가변길이 헤더의 경우 헤더길이를 알려주는 HLEN 필드가 있어야 한다.)

- 플래그를 보고 데이터 전송 전인지, 중인지, 후인지를 알 수 있다.
    - syn이 1이면 전송 전
    - fin이 1이면 전송 후
    - 나머지는 전송 중에 관여

### TCP 연결 설정
- 클라이언트가 서버에게 목적지 포트번호를 포함하여 전송(80번 열려 있니?)
    - 서버가 SYN을 받았으니 ACK를 보냄(잘 받았다.)
    - 서버의 80번 포트가 열러 있으니 클라이언트에게 준비가 되었는지 확인(SYN 전송)
    - 정상적으로 서비스가 열려있지 않으면 SYN 대신에 RST(Reset) 전송
- 클라이언트가 80번 포트가 열려 있는지에 대한 ACK를 전송

- **연결을 시도하는 쪽이 클라이언트**

- 연결 설정 중의 SN, AN
    1. 클라이언트가 서버에게 SYN 1, SN을 보낸다.(SN은 랜덤 초기화 값: ex100)
    2. 서버가 ACK 1, SN + 1로 AN(101), SYN 1, SN(랜덤 초기화 값: ex300)을 보낸다.
    3. 클라이언트가 SYN은 0, SN은 이전 단계의 AN인 101, ACK는 1, AN은 SN + 1인 301을 보낸다.

    

### 트래픽 전송 과정
- 바이트 단위 전송.

- 정상적인 전송 과정
    - 요청받은 만큼 데이터를 전송. 전송이 되면 받았다는 AN을 보낸다.
    - 그러면 송신 호스트는 다음 데이터를 전송한다.
    - **AN = SN + DataSize**
- 비정상적 전송 과정
    - 받은 만큼 SN에 더하여 AN을 보낸다. 만약 AN이 정상적으로 오지 않으면 중간에 데이터가 누락되었다고 판단하고 다시 보낸다.

    - 헤더에는 몇 바이트를 전송하는지 표기하지 않는다.(DataSize 표기 X)
    - **AN으로 신뢰성을 확보한다.** 전송의 성공 유무 판단 

    - 데이터를 보냈는데 AN이 돌아오지 않는 경우도 존재
    - RTT * 2정도의 시간을 체크하여 재전송한다.
        - RTT: Round Trip Time, 패킷이 갔다오는 시간. 3WHS 때 측정

### 트래픽 흐름제어
- **흐름 제어**: 수신측의 상태에 따라 데이터 전송량 조절 --> 윈도우 사이즈
- TCP의 느린 전송 속도를 보완하기 위한 기술
- 윈도우 사이즈에 맞게 **전송량 조절**
    - 받는 쪽에서 보내는 쪽에 사이즈를 알려준다.
    - 연속해서 받을 수 있는 데이터 사이즈
    - 윈도우 사이즈도 3WHS때 결정
    - 데이터 전송 중에도 조절 가능
    - RAM이나 CPU에 따라 조절

- 윈도우 사이즈 0을 보냈을 때
    - 데이터 송신 중단 --> 일정 시간 대기 후 Keep Alive(전송 재개 요청)으로 0이 아닌 값을 받으면 응답을 받음. 
        - **만약 응답이 오지 않는다면 세션 종료**
        - **제로 윈도우 사이즈가 지속되는 것은 심각한 문제가 될 수 있다.**

- 윈도우 사이즈 값에 맞게 보내다 사이즈가 갑자기 커졌을 때
    - 리소스 여유가 있다는 뜻
    - 더 많은 데이터 전송 


- 중간에 데이터 누락이 존재할 경우
    - 수신자도 데이터 누락을 알 수 있다.(SN,AN을 통해)
    - 수신자가 NAK 신호를 보냄 --> 누락된 데이터 재전송 요청
        - 재전송 요청이 3번이 가면 즉시 그 데이터를 보내준다. (Fast Retransmission)

    - 디폴트는 보내던 데이터셋을 모두 보내지만, 3WHS 시 정할 수 있다.
        - TCP Option의 **SACK(selective ACK)** permitted: 문제가 있는 것만 보냄(permitted된 경우)
        - TCP Option의 **window scale**: 윈도우 사이즈 크기를 한 번에 몇 개 처리할 수 있는지를 알려주는 옵션

- 데이터에 대한 대답은 TCP로 보낸다.(ACK 포함)


    ### 정리
    - TCP Window 제어와 Retransimisson 방식
        1. ACk가 분실되는 경우
            - 데이터가 잘 전송되었어도 보내는 쪽이 인지하지 못하므로 기존의 방식은 데이터를 재전송해야 한다.
            - 윈도우 방식을 사용하면 ACK가 일부 분실되어도 그 다음 세그먼트까지 잘 받았다는 ACK를 받기에 재전송을 하지 않아도 된다.

        2. 보내는 세그먼트가 분실되는 경우
            - 데이터가 분실되면 동일한 번호로 된 ACK를 연속으로 반환한다.
            - 송신 호스트측에서는 ACK를 연속 3번 더 수신하는 경우 ACK가 말하는 Data Segment를 재전송한다.
            - 이 과정은 timeout으로 인한 retransmission보다 빠르게 일어나므로, Fast Retramsmission이라고 한다.

### 연결 종료
- 4-way handshake
- FIN 플래그 1인 경우 연결 종료 집입.
- 클라이언트-서버 관계에서의 예시.(**양단간에도 가능**하다.)
    1. 클라이언트가 FIN을 보낸다.
    2. 서버에서 ACK신호를 보내고, 클라이언트에 할당했던 프로세스를 종료한다.(close_wait)
        - CLOSE_WAIT에서 LAST_ACK로 넘어가지 못하면 애플리케이션에 문제가 있는 것이다.
    3. 서버가 모든 애플리케이션을 종료 후 LAST_ACK와 FIN을 보낸다.
    4. ACK를 보낸다.
    5. 서버가 종료한다.
    6. 클라이언트가 종료한다.
    - 가상 회선 끊어짐 --> 데이터 전송 X

- 서버보다 클라이언트가 먼저 끊어진 경우 발생할 수 있는 문제
    - 클라이언트가 ACK를 보내면서 세션을 끊은 경우 
        - 서버에서 클라이언트로 향하는 세션만 남아있는 상태.
        - 이 상태에서 **클라이언트가 보낸 ACK가 소실될 경우**
            - 서버는 FIN에 대한 ACK를 못받아 계속해서 FIN을 보낸다.
            - 서버에서는 소켓을 종료하지 못하게 됨.
            - 이를 방지하기 위한 것이 **TIME_WAIT**
                - 혹시라도 FIN이 두 번 올 경우를 대비하여 대기하는 시간.
                - 대략 RTT * 2 정도의 시간(2MSL: Max Segment Lifetime)
        






### 정리
- TCP 역할
    - 데이터 전송 전: 연결성 지향
    - 데이터 전송 중: 신뢰성 확인, 무결성 확인, 흐름제어
    - 데이터 전송 후: 연결 종료

## UDP
### UDP 헤더
- TCP와는 다르게 Data 전송을 위하여 사전에 필요한 Process가 없음
- Best-Effort-Delivery
- 신뢰성 보장 못함
- 헤더 크기도 작다.(TCP에 비해 간단하다.)


### UDP와 TCP
- UDP
    - 비연결지향적: 데이터 전송 전 상태 확인 없음. 
    - 신뢰성 X: 전송의 성공/실패 여부에 상관 없이 재전송 하지 않음
    - 오류 제어 기능 없음: 체크섬으로 오류가 점검 되더라도 재전송 요청하지 않음
    - 흐름 제어 기능 없음
    - 단방향 전송
    - 실시간 멀티 미디어 데이터 전송 시 사용
    - Message-Oriented Transport Protocol
    - 상위계층에서 payload된 데이터를 단편화하지 않음
    - 데이터 전송이 완료되기 전까지는 **다른 데이터 흐름에 영향을 받지 않음**
    - 상태 체크를 하지 않아 **항상 데이터를 포함**하고 있음
    - 확인 작업 없음
    - **고정 길이 헤더 사용(8byte)**
        - UDP Length는 데이터를 포함한 전체 길이(total length)
        - UDP Checksum은 오류를 검출해도 애플리케이션에서 처리하고, 재전송을 요청하지는 않는다.
        - UDP Length가 39라면, UDP헤더를 포함한 데이터 세그먼트의 크기가 39바이트고, 페이로드된 데이터의 크기는 31바이트임을 알 수 있다.

- TCP
    - 연결지향적: 서로간의 상태를 확인해서 정상적으로 활성화가 되어 있어야만 데이터 전송을 시도
    - 신뢰성: 전송이 성공할 때까지 재전송. (전송을 보장한다. ACK) 
    - 흐름 제어: 송수신되는 데이터 양을 조절(WS)
    - 오류 제어: 체크섬을 통해서 무결성 점검 가능 --> 무결성 깨질 시 재전송 요청(Checksum)
    - 양방향 전송
    - 일반적인 데이터 전송(신뢰성이 중요한 유니캐스트 데이터 전송 시)
    - Stream-Oriented Transtport Protocol
    - 상위계층에서 payload 된 데이터를 단편화
    - 단편화를 통해 다수의 데이터들과 네트워크 자원을 

## IP
### IP Header
- IP헤더는 총 20Byte의 기본 길이와 옵션을 사용해 크기가 최대 60Byte까지 가질 수 있다.
- IP는 **항상 데이터를 싣고 가야 한다.** 페이로드가 되지 않으면 쓰레기값을 만들어서라도 채워서 보내야 한다.
    - ping은 3계층 테스트. 데이터가 없지만 강제로라도 넣어서 보낸다.
- 가변 길이 헤더 --> Header Length 필드가 반드시 필요하다.

- 헤더의 필드:
- VER(version) 필드: IP 프로토콜의 버전(1~6) 4bit

- HLEN(Header Length) 필드: 4bit

- Service type()필드: 전송하려는 데이터가 느려도 안전하게 전송할것인지, 딜레이 없이 빠르게 전송할 것인지에 대한 **요구사항을 넣을 수 있는 필드.**(QoS필드. 영상이나 이미지일 때 사용 6bit,2bit 합쳐서 8bit)

- Total length:  IP헤더와 Payload를 포함한 바이트 단위 길이. 16bit

- **단편화** 작업 관련 필드
    - Identification: SN과 비슷함. 단편화 시 사용되는 식별자. 16bit
    - Flags: 첫 번째 비트는 사용하지 않음. 나머지 두 비트만 사용.
        - DF (Don’t Fragment): 단편화 금지 플래그
        - MF (More Fragment): 이 패킷 이후 추가 단편이 있음을 알리는 플래그
        - 0이면 마지막 플래그
        - 첫 단편은 플래그 1(001), 오프셋 0
        - 3bit
    - Fregmentation offset: 단편을 조립해 한 데이터로 만들 수 있도록 단편의 위치를 기술한 정보. 13bit

- TTL(Time To Live)
    - 패킷 수명을 제한하기 위해 데이터그램이 통과하는 **최대 홉(hop)수**를 지정
    - 패킷 전달과정에서 라우터와 같은 전송장비를 통과 할 때마다 TTL값 감소
    - TTL이 0이 되면 라우터에서 폐기하여 불필요한 패킷이 네트워크에 방치 되는 것을 방지
    -  OS종류와 버전에 따라 TTL 값이 다름(변경 가능)
    - 게이트웨이까지는 감소하지 않음.(L2 장비는 감소X)
    - TTL이 너무 작으면 목적지에 도달하기 전에 폐기된다.
    - TTL이 너무 크면 목적지가 다운됐을 때에도 존재하지 않는 목적지를 찾아 헤매는 일이 발생한다.
    - 8bit

- Protocol: 현재 어떤 프로토콜을 싣고 가는지 명시하는 부분. 8bit

- Header checksum: 오류 검출. 16bit

- 송신지IP: 32bit

- 수신지IP: 32bit

### IP 특징
- MTU: LAN카드 때문에 나온 개념. LAN카드가 한 번에 밖으로 내보낼 수 있는 데이터의 양이 한정되어 있기 때문에 조절을 해야 한다. (한 번에 저장할 수 있는 버퍼가 정해져있어 그 이상을 버려버린다.)
    - 일반적인 이더넷v2는 1500 바이트를 지원한다.
    - 4계층에서 단편화는 선택적이지만, 4계층에서 안하면 3계층에서 필수적으로 해주어야 한다.
- 송신지에서 분할하는 방법(경로 MTU 탐색)
    - 라우터에서 ICMP 메시지를 통해 MTU를 알려줌.
    - 송신지에서 분할하여 송신
- 단편화일 경우 Identification 값이 같을 수 있다.(일반적으로는 문제가 된다.)
- 단편화: 데이터 분할(송신지)
- 재조립: 데이터 병합(수신지)
    - 재조립 시 공격하는 해킹 기법들도 존재. 
    - 분할보다 재조립이 오래 걸린다.

- SAP: 무엇을 싣고 가는지 명시해주는 것. TCP에서는 목적지 포트 번호가 SAP가 된다.(포트 번호를 통해 무엇을 위한 패킷인지를 알 수 있음.)

# 와이어샤크
## 스위치 환경에서의 스니핑 방법
- 스위치의 경우 포워딩과 플러딩 방식을 사용하고, 유니캐스트 패킷은 주로 포워딩을 사용하기 때문에 sniffer로 잘 넘어가지 않는다.
- 이를 보기 위해선 관제 범위를 넓혀야 하는데, 3가지 방법이 있다.


### 포트 미러링
- 스위치의 소스에서 들어오고 나가는 모든 데이터를 목적지로 복사해서 전달하는 역할.
- 타깃과 같은 데이터를 받는다.
- 특정 포트로 들어오는 모든 데이터를 Sniffer도 받는다.
- 스위치 담당자가 설정해주어야 한다.

### 허빙 아웃
- 스위치 밑에 허브를 달아놓고 타깃을 허브에 연결한다.
    - 이런 경우는 많지 않다.
    - 보통 학습용으로 사용하는 경우가 대부분이다.
    - 회사에서는 스위치 담당자 등에게 말을 해야 한다.

### 탭 사용
- 허빙 아웃 방식에서 허브 대신 TAP이라는 장비를 사용하는 것
- TAP(Test Access Ports): 데이터 흐름의 중단 없이 네트워크에 전혀 영향을 주지 않으면서 트래픽을 모디터링 할 수 있게 해주는 모니터링 장비
- 최근 네트워크 탭은 스위치처럼 12포트, 24포트가 있어 많은 수의 장비를 동시에 사용 가능

- Aggregation TAP
    - 1개 이상의 네트워크 링크 또는 포트에서 수집한 패킷 데이터들을, 1개의 NIC를 사용하는 모디터 또는 그 이상의 여러 모니터 장비로 보내 분석할 수 있게 해 주는 탭

### 정보 수집 방법
- 지연이 발생한 호스트에 와이어 샤크를 설치하여 **직접 수집**

- 호스트의 스위치 포트를 확장(포트 미러링)
    - 스위치 설정 필수
- TAP 설정
    - 포트 미러링 필요 X
    - 관제사의 PC를 연결하여 수집


## 와이어샤크 필터
1. 캡처 필터
    - 패킷이 캡처될 때 지정(수집 대상을 지정)
    - 지정된 표현식에 포함/제외된 패킷만 캡처

2. **디스플레이 필터**
    - 원하지 않는 패킷을 숨김
    - 지정된 표현식을 기반으로 원하는 패킷을 보기
    - 원하는 패킷 종류(3WHS등)만 뽑아낼 수 있다.
    - 패킷 골라내기!!

- 필터링: 원하는 정보만 보는 작업